<a href="https://colab.research.google.com/github/427Stanley427/Python/blob/main/cl%C3%A9s_BTC_d%C3%A9filement_automatique_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bitcoin

In [ ]:
pip install requests

In [ ]:
import requests
import bitcoin
import random

def generate_bitcoin_keypair_with_prefix():
    while True:
        private_key = bitcoin.random_key()
        public_key_uncompressed = bitcoin.privtopub(private_key)

        # Décider du type d'adresse en fonction de la probabilité
        r = random.random()  # Génère un nombre aléatoire entre 0 et 1
        if r < 0.3:
            prefix = "1"
            address = bitcoin.pubtoaddr(public_key_uncompressed)
            address_type = "P2PKH"
        elif 0.3 <= r < 0.6:
            prefix = "3"
            address = bitcoin.pubtoaddr(public_key_uncompressed)
            address_type = "P2SH"
        else:
            prefix = "bc1"
            address = bitcoin.pubtosegwitaddr(public_key_uncompressed)
            address_type = "SegWit P2WPKH"

        print(f"Vérification de l'adresse ({address_type}): {address}")
        balance = get_balance(address)

        if balance > 0:
            wif_private_key = bitcoin.encode_privkey(private_key, 'wif')
            return {
                "private_key_hex": private_key,
                "wif_private_key": wif_private_key,
                "address_type": address_type,
                "address": address,
                "balance": balance
            }

def get_balance(address):
    url = f"https://api.blockchair.com/bitcoin/dashboards/address/{address}"
    response = requests.get(url)

    # Vérifier si la requête a réussi
    if response.status_code != 200:
        print(f"Erreur lors de la récupération du solde pour l'adresse {address}: {response.text}")
        return 0

    try:
        data = response.json().get("data", {})
    except requests.exceptions.JSONDecodeError:
        print("Erreur lors de la décodage JSON. Réponse non valide ou vide reçue.")
        return 0

    address_data = data.get(address, {})
    balance_satoshis = address_data.get("address", {}).get("balance", 0)
    return balance_satoshis / 100_000_000  # Convertir en BTC

kp = generate_bitcoin_keypair_with_prefix()

print("\nAdresse avec balance positive trouvée!")
print("Clé privée hex :", kp["private_key_hex"])
print("Clé privée WIF :", kp["wif_private_key"])
print(f"Adresse ({kp['address_type']}) :", kp["address"], "| Balance:", kp["balance"], "BTC")
